In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Now import other libraries or define your model



In [1]:
import ctypes
try:
    ctypes.WinDLL('zlibwapi.dll')  # Test loading the DLL
    print("Success: zlibwapi.dll is accessible!")
except Exception as e:
    print(f"Error: {e}")

Error: [WinError 193] %1 is not a valid Win32 application


In [2]:
import os
import numpy as np
from PIL import Image
import pandas as pd
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras.mixed_precision import LossScaleOptimizer

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
set_global_policy('mixed_float16')



INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA RTX A4000, compute capability 8.6


In [3]:
from keras import backend as K

# Clear previous sessions
K.clear_session()


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:

# Load metadata
metadata_path = "Dataset/meta_train.csv"
metadata_df = pd.read_csv(metadata_path)
metadata_df.set_index('image_id', inplace=True)

# Path to preprocessed folder
base_path = "Dataset/preprocessed_images"
diseases = os.listdir(base_path)

X = []
y = []

for disease in diseases:
    disease_path = os.path.join(base_path, disease)
    files = os.listdir(disease_path)

    base_names = sorted(set(f.split('_')[0] for f in files))

    for base in base_names:
        try:
            # Construct paths
            red_path = os.path.join(disease_path, f"{base}_red.jpg")
            green_path = os.path.join(disease_path, f"{base}_green.jpg")
            blue_path = os.path.join(disease_path, f"{base}_blue.jpg")


            if not all(os.path.exists(p) for p in [red_path, green_path, blue_path]):
                print(f"Skipping {base}: not all image variants found.")
                continue

            # Load and resize each channel as grayscale
            red = np.array(Image.open(red_path).resize((128, 128)).convert('L'))
            green = np.array(Image.open(green_path).resize((128, 128)).convert('L'))
            blue = np.array(Image.open(blue_path).resize((128, 128)).convert('L'))


            # Now stack them into (299, 299, 4)
            stacked = np.stack([red, green, blue], axis=-1).astype(np.float32)
            stacked /= 255.0  # Normalize

            # Check metadata
            meta_key = f"{base}.jpg"
            if meta_key not in metadata_df.index:
                print(f"Metadata not found for {meta_key}")
                continue

            age = float(metadata_df.loc[meta_key, 'age'])

            X.append(stacked)
            y.append(age)

        except Exception as e:
            print(f"Error processing {base}: {e}")

# Convert to numpy arrays



In [6]:
X = np.array(X)
y = np.array(y)

In [7]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (10407, 224, 224, 3)
y shape: (10407,)


In [ ]:
# base_model = InceptionV3(
#     input_shape=(299, 299, 3),
#     include_top=False,
#     weights=None,
#     pooling='avg'
# )
# Define input
# input_tensor = Input(shape=(299, 299, 3))

# # Load EfficientNetB0 without pretrained weights
# base_model = EfficientNetB0(input_tensor=input_tensor, include_top=False, weights=None, pooling='avg')


: 

In [ ]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define input layer
input_tensor = Input(shape=(128, 128, 3))

# InceptionV3 with no weights
base_model = EfficientNetB0(include_top=False, weights=None, input_tensor=input_tensor)

# Regression head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='linear', dtype='float32')(x)
optimizer = Adam(learning_rate=0.001)
optimizer = LossScaleOptimizer(optimizer)  # Wraps the optimizer

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# Train
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, batch_size=2)


Epoch 1/2


In [ ]:
# loss, mae = model.evaluate(X_val, y_val)
# print("Validation MAE:", mae)

In [ ]:
loss, mae = base_model.evaluate(X_val, y_val)
print("Validation MAE:", mae)